<a href="https://colab.research.google.com/github/Sirin99-sc/Introduction_to_Deep_Learning_and_Neural_Networks_Coursera/blob/main/Week%205/Peer_graded_Assignment_Build_a_Regression_Model_in_Keras_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regression Models With Keras**

## **Download and Clean Dataset**
Let's start by importing the pandas and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be using the dataset provided in the assignment

**The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:**

  **1. Cement**

  **2. Blast Furnace Slag**

**3. Fly Ash**

**4. Water**

**5. Superplasticizer**

**6. Coarse Aggregate**

**7. Fine Aggregate**

Let's download the data and read it into a *pandas* dataframe.



In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.

### Let's check how many data points we have

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to n_cols since we will need this number when building our network.

In [9]:
n_cols = predictors.shape[1]
n_cols

8

## **Import Keras**

### Let's go ahead and import the Keras library

In [10]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [11]:
from keras.models import Sequential
from keras.layers import Dense

## **Build a Neural Network**

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [13]:
from sklearn.model_selection import train_test_split

Let's split the data into a training and test sets by holding 30% of the data for testing

In [14]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

## **Train and Test the Network**

Let's call the function now to create our model.

In [15]:
# build the model
model = regression_model()

Next, we will train the model for 50 epochs.

In [27]:
# fit the model
epochs=50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
23/23 [==============================] - 0s 2ms/step - loss: 46.3977
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 45.8636
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 46.4478
Epoch 4/50
23/23 [==============================] - 0s 3ms/step - loss: 46.9933
Epoch 5/50
23/23 [==============================] - 0s 2ms/step - loss: 45.7120
Epoch 6/50
23/23 [==============================] - 0s 2ms/step - loss: 45.8056
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 46.3376
Epoch 8/50
23/23 [==============================] - 0s 2ms/step - loss: 45.9516
Epoch 9/50
23/23 [==============================] - 0s 2ms/step - loss: 45.9828
Epoch 10/50
23/23 [==============================] - 0s 2ms/step - loss: 45.3933
Epoch 11/50
23/23 [==============================] - 0s 2ms/step - loss: 45.2245
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 45.2528
Epoch 13/50
23/23 [==================

Next we need to evaluate the model on the test data.

In [28]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 2ms/step


56.115543365478516

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [22]:
from sklearn.metrics import mean_squared_error

In [29]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

56.11553774387635 0.0


Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [30]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []

for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 47.35017013549805
10/10 [==============================] - 0s 2ms/step
MSE 2: 51.573726654052734
10/10 [==============================] - 0s 2ms/step
MSE 3: 41.469993591308594
10/10 [==============================] - 0s 2ms/step
MSE 4: 47.35120391845703
10/10 [==============================] - 0s 3ms/step
MSE 5: 47.785526275634766
10/10 [==============================] - 0s 2ms/step
MSE 6: 47.46910858154297
10/10 [==============================] - 0s 2ms/step
MSE 7: 54.25178146362305
10/10 [==============================] - 0s 2ms/step
MSE 8: 47.24246597290039
10/10 [==============================] - 0s 2ms/step
MSE 9: 44.12272644042969
10/10 [==============================] - 0s 2ms/step
MSE 10: 47.77294158935547
10/10 [==============================] - 0s 2ms/step
MSE 11: 44.452125549316406
10/10 [==============================] - 0s 2ms/step
MSE 12: 42.23789978027344
10/10 [==============================] - 0s 2ms/step
MSE 13: 51.19456481933594
10/10 [========================